<h1>Indexing and selecting data</h1>

<p>The axis labeling information in pandas objects serves many purposes:</p>

<ul>
<li><p>Identifies data (i.e. provides <em>metadata</em>) using known indicators,
important for analysis, visualization, and interactive console display.</p></li>
<li><p>Enables automatic and explicit data alignment.</p></li>
<li><p>Allows intuitive getting and setting of subsets of the data set.</p></li>
</ul>

<p>In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects.
The primary focus will be on Series and DataFrame as they have received more development attention in this area.</p>

<p>Note</p>

<p>The Python and NumPy indexing operators <code>[]</code> and attribute operator <code>.</code> provide quick and easy access to pandas data structures across a wide range of use cases.
This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays.
However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits.
For production code, we recommended that you take advantage of the optimized
pandas data access methods exposed in this chapter.</p>

<p>Warning</p>

<p>Whether a copy or a reference is returned for a setting operation, may
depend on the context. This is sometimes called <em>chained assignment</em> and
should be avoided. See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy">Returning a View versus Copy</a>.</p>

<p>See the <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced">MultiIndex / Advanced Indexing</a> for <code>MultiIndex</code> and more advanced indexing documentation.</p>

<p>See the <a href="https://pandas.pydata.org/docs/user_guide/cookbook.html#cookbook-selection">cookbook</a> for some advanced strategies.</p>

<h2>Different choices for indexing</h2>

<p>Object selection has had a number of user-requested additions in order to
support more explicit location based indexing. Pandas now supports three types
of multi-axis indexing.</p>

<ul>
<li><p><code>.loc</code> is primarily label based, but may also be used with a boolean array. <code>.loc</code> will raise <code>KeyError</code> when the items are not found. Allowed inputs are:</p>

<blockquote>
<div><ul>

<li><p>A single label, e.g. <code>5</code> or <code>'a'</code> (Note that <code>5</code> is interpreted as a <em>label</em> of the index. This use is <strong>not</strong> an integer position along the index.).</p></li>

<li><p>A list or array of labels <code>['a', 'b', 'c']</code>.</p></li>

<li><p>A slice object with labels <code>'a':'f'</code> (Note that contrary to usual Python slices, <strong>both</strong> the start and the stop are included, when present in the index! See <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-slicing-with-labels">Slicing with labels</a>
and <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-endpoints-are-inclusive">Endpoints are inclusive</a>.)</p></li>

<li><p>A boolean array (any <code>NA</code> values will be treated as <code>False</code>).</p></li>

<li><p>A <code>callable</code> function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).</p></li>

<li><p>A tuple of row (and column) indices whose elements are one of the
above inputs.</p></li>

</ul>
</div></blockquote>

<p>See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-label">Selection by Label</a>.</p>
</li>

<li><p><code>.iloc</code> is primarily integer position based (from <code>0</code> to <code>length-1</code> of the axis), but may also be used with a boolean
array.
<code>.iloc</span></code> will raise <code>IndexError</code> if a requested
indexer is out-of-bounds, except <em>slice</em> indexers which allow
out-of-bounds indexing.  (this conforms with Python/NumPy <em>slice</em>
semantics).  Allowed inputs are:</p>

<blockquote>
<div><ul>

<li><p>An integer e.g. <code>5</code>.</p></li>

<li><p>A list or array of integers <code>[4,</span> <span class="pre">3,</span> <span class="pre">0]</span></code>.</p></li>

<li><p>A slice object with ints <code>1:7</span></code>.</p></li>
<li><p>A boolean array (any <code>NA</span></code> values will be treated as <code>False</code>).</p></li>

<li><p>A <code>callable</code> function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).</p></li>

<li><p>A tuple of row (and column) indices whose elements are one of the
above inputs.</p></li>

</ul>
</div></blockquote>

<p>See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-integer">Selection by Position</a>, <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced">Advanced Indexing</a> and <a href="https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-advanced-hierarchical">Advanced Hierarchical</a>.</p>
</li>

<li><p><code>.loc</code>, <code>.iloc</code>, and also <code>[]</code> indexing can accept a <code>callable</code> as indexer. See more at <a href="https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-callable">Selection By Callable</a>.</p>

<div>
<p>Note</p>

<p>Destructuring tuple keys into row (and column) indexes occurs
<em>before</em> callables are applied, so you cannot return a tuple from
a callable to index both rows and columns.</p>
</div>
</li>
</ul>

<p>Getting values from an object with multi-axes selection uses the following
notation (using <code>.loc</code> as an example, but the following applies to <code>.iloc</code> as well).
Any of the axes accessors may be the null slice <code>:</code>.
Axes left out of the specification are assumed to be <code>:</code>, e.g. <code>p.loc['a']</code> is equivalent to <code>p.loc['a', :]</code>.</p>

In [6]:
import pandas as pd
import numpy as np

In [3]:
ser = pd.Series(range(5), index=list("abcde"))

In [4]:
ser.loc[["a", "c", "e"]]

,0
a,0
c,2
e,4


In [7]:
df = pd.DataFrame(np.arange(25).reshape(5, 5), index=list("abcde"), columns=list("abcde"))

In [8]:
df.loc[["a", "c", "e"], ["b", "d"]]

,b,d
a,1,3
c,11,13
e,21,23


<h2>Basics</h2>

<p>As mentioned when introducing the data structures in the last section, the primary function of indexing with <code>[]</code> (a.k.a. <code>__getitem__</code> for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices.
The following table shows return type values when indexing pandas objects with <code>[]</code>:</p>

<table>
<colgroup>
<col style="width: 25.0%">
<col style="width: 25.0%">
<col style="width: 50.0%">
</colgroup>
<thead>
<tr><th><p>Object Type</p></th>
<th><p>Selection</p></th>
<th><p>Return Value Type</p></th>
</tr>
</thead>
<tbody>
<tr><td><p>Series</p></td>
<td><p><code>series[label]</span></code></p></td>
<td><p>scalar value</p></td>
</tr>
<tr><td><p>DataFrame</p></td>
<td><p><code>frame[colname]</span></code></p></td>
<td><p><code>Series</span></code> corresponding to colname</p></td>
</tr>
</tbody>
</table>

<p>Here we construct a simple time series data set to use for illustrating the
indexing functionality:</p>